# Tool Pattern
The agent uses external tools (e.g., APIs, databases, or file systems) to enhance its functionality.

<img src = "https://github.com/neural-maze/agentic_patterns/raw/main/img/tool_pattern.png" height ="650">

In [18]:
import json

def get_weather_data(location : str , unit : str):
    if location == "Madrid":
        return json.dumps({"temperature":25,"unit":unit})
    else:
        return json.dumps({"temperature":25,"unit":unit})

In [19]:
get_weather_data(location="Madrid", unit="celsius")

'{"temperature": 25, "unit": "celsius"}'

In [20]:
import os
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
client = Groq(api_key=GROQ_API_KEY)

In [21]:
model = "llama-3.3-70b-versatile"

tool_system_prompt = """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. 
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug 
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

Here are the available tools:

<tools> {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location location (str): The city and state, e.g. Madrid, Barcelona unit (str): The unit. It can take two values; 'celsius', 'fahrenheit'",
    "parameters": {
        "properties": {
            "location": {
                "type": "str"
            },
            "unit": {
                "type": "str"
            }
        }
    }
}
</tools>
"""

In [22]:
tool_chat_history = [
    {
        "role" : "system",
        "content": tool_system_prompt
    }
]

agent_chat_history = []

user_msg = {
    "role" : "user",
    "content" : "What is the Current Temperature in Madrid in Celsius"
}

tool_chat_history.append(user_msg)
agent_chat_history.append(user_msg)

response = client.chat.completions.create(
    model = model ,
    messages= tool_chat_history
)

response = response.choices[0].message.content

print("LLM Response :",response)

LLM Response : <tool_call>
{"name": "get_current_weather", "arguments": {"location": "Madrid", "unit": "celsius"}}
</tool_call>


In [23]:
import re
import json

def parse_tool_call_str(tool_call_str: str):
    # Extract JSON inside <tool_call> tags
    pattern = r'<tool_call>(.*?)</tool_call>'
    match = re.search(pattern, tool_call_str, re.DOTALL)

    if not match:
        print("No valid JSON found in tool_call tags.")
        return None

    clean_json = match.group(1).strip()  # Extract matched JSON content

    try:
        return json.loads(clean_json)  # Parse JSON
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")
        return None


In [24]:
parsed_output = parse_tool_call_str(response)

# Mock function
def get_weather_data(location, unit):
    return {"temperature": 18, "unit": unit, "location": location}

# Execute if parsing is successful
if parsed_output and "arguments" in parsed_output:
    result = get_weather_data(**parsed_output["arguments"])
    print(parsed_output["arguments"])  
    print(result) 

{'location': 'Madrid', 'unit': 'celsius'}
{'temperature': 18, 'unit': 'celsius', 'location': 'Madrid'}


In [25]:
result = get_weather_data(**parsed_output["arguments"])
result

{'temperature': 18, 'unit': 'celsius', 'location': 'Madrid'}

In [26]:
agent_chat_history.append(
    {
        "role" : "user",
        "content" : f"Observation: {result}"
    }
)

In [27]:
response = client.chat.completions.create(
    messages= agent_chat_history,
    model = model
)

response = response.choices[0].message.content

In [28]:
import json
import requests
from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.tool_pattern.tool_agent import ToolAgent


def fetch_top_hacker_news_Stories(top_k : int):
    
    top_stories_url = 'https://hacker-news.firebaseio.com/v0/topstories.json'
    try : 
        response = requests.get(top_stories_url)
        response.raise_for_status()

        top_story_ids = response.json()[:top_k]

        top_Stories = []

        for story_id in top_story_ids:
            story_url = f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json'
            story_response = requests.get(story_url)
            story_response.raise_for_status()  
            story_data = story_response.json()

            top_Stories.append(
                {
                    "title": story_data.get('title','No title'),
                    "url" : story_data.get('url','No URL Available')
                }
            )
        return json.dumps(top_Stories)
    except requests.exceptions.RequestException as e:
        print(f"Error : {e}")
        return  []


In [29]:
json.loads(fetch_top_hacker_news_Stories(top_k=5))

[{'title': 'Let’s code a TCP/IP stack, 1: Ethernet and ARP (2016)',
  'url': 'https://www.saminiir.com/lets-code-tcp-ip-stack-1-ethernet-arp/'},
 {'title': 'Repairable Flatpack Toaster',
  'url': 'https://www.kaseyhou.com/#/repairable-flatpack-toaster/'},
 {'title': 'Windows NT for GameCube/Wii',
  'url': 'https://github.com/Wack0/entii-for-workcubes'},
 {'title': "The cost of Go's panic and recover",
  'url': 'https://jub0bs.com/posts/2025-02-28-cost-of-panic-recover/'},
 {'title': "Apple's Software Quality Crisis",
  'url': 'https://www.eliseomartelli.it/blog/2025-03-02-apple-quality'}]

In [30]:
hn_tool = tool(fetch_top_hacker_news_Stories)

In [31]:
hn_tool.name

'fetch_top_hacker_news_Stories'

In [32]:
json.loads(hn_tool.fn_signature)

{'name': 'fetch_top_hacker_news_Stories',
 'description': None,
 'parameters': {'properties': {'top_k': {'type': 'int'}}}}

## Tool Agent

In [34]:
tool_agent = ToolAgent(tools =[hn_tool],model = model)

response = tool_agent.run(user_msg="Who are you ")
response

'I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."'

In [36]:
response = tool_agent.run(user_msg="Tell me the top 5 Hacker News stories right now")
print(response)


Using Tool: fetch_top_hacker_news_Stories

Tool call dict: 
{'name': 'fetch_top_hacker_news_Stories', 'arguments': {'top_k': 5}, 'id': 1}

Tool result: 
[{"title": "Let\u2019s code a TCP/IP stack, 1: Ethernet and ARP (2016)", "url": "https://www.saminiir.com/lets-code-tcp-ip-stack-1-ethernet-arp/"}, {"title": "Repairable Flatpack Toaster", "url": "https://www.kaseyhou.com/#/repairable-flatpack-toaster/"}, {"title": "Windows NT for GameCube/Wii", "url": "https://github.com/Wack0/entii-for-workcubes"}, {"title": "The cost of Go's panic and recover", "url": "https://jub0bs.com/posts/2025-02-28-cost-of-panic-recover/"}, {"title": "Apple's Software Quality Crisis", "url": "https://www.eliseomartelli.it/blog/2025-03-02-apple-quality"}]
Based on the provided data, the top 5 Hacker News stories right now are:

1. **Let’s code a TCP/IP stack, 1: Ethernet and ARP (2016)** - https://www.saminiir.com/lets-code-tcp-ip-stack-1-ethernet-arp/
2. **Repairable Flatpack Toaster** - https://www.kaseyhou.